In [120]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
from analyze_budget import analyze_multiple
import pandas as pd

In [122]:
fedavg_path = 'new_topology_experiment_results/jorge_setting_fedavg/comm_freq_5'

fc_fed_path = "experiment_results/jorge_setting_fedavg"

In [123]:
# metric = 'auc'
metric = 'final_acc'
error_type = 'sem'

In [124]:
# analyze_multiple(fedavg_path)

In [125]:
def cal_mean_and_error(df, keys=None, metric=None,
                      error_type="sem"):
   if keys is None:
      keys = ["algo", "use_contrastive"]
   if metric is None:
      metric = "final_acc"
   keys += ["dataset"]
   m = df.groupby(keys)[
         metric].mean()
   if error_type == "sem":
      stderr = df.groupby(keys)[metric].sem()
   else:
      stderr = df.groupby(keys)[metric].std()
   df = pd.DataFrame(m).reset_index()
   df[metric + "_error"] = stderr.values
   return df

In [126]:
def normalize_by_baseline(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_normalized = df.copy()
    
    # Identify the baseline values and errors for modular and monolithic
    baseline = df_normalized[df_normalized['algo'].str.contains('baseline')].set_index(['dataset', 'algo'])
    baseline_auc = baseline[metric]
    baseline_error = baseline[metric + '_error']

    # Function to apply normalization and error calculation
    def normalize_row(row):
        baseline_key = (row['dataset'], 'modular (baseline)' if 'modular' in row['algo'] else 'monolithic (baseline)')
        norm_value = (row[metric] - baseline_auc[baseline_key]) / baseline_auc[baseline_key]
        # norm_error = (row[metric] + row[metric + '_error'] - baseline_auc[baseline_key])  / baseline_auc[baseline_key]  # Assuming negligible error in baseline
        norm_error = row[metric + '_error'] / baseline_auc[baseline_key]  # Assuming negligible error in baseline

        return pd.Series([norm_value, norm_error], index=[metric, metric + '_error'])

    # Apply normalization and error calculation
    df_normalized[[metric, metric + '_error']] = df_normalized.apply(normalize_row, axis=1)
    return df_normalized

def map_datasets_to_new_groups(df):
    """Map original dataset names to new group names 'homo' or 'hetero'."""
    dataset_group_map = {
        'mnist': 'homogenous', 'fashionmnist': 'homogenous', 'kmnist': 'homogenous',
        # 'cifar100': 'heterogeneous', 'combined': 'heterogeneous'
        'cifar100': 'cifar100', 'combined': 'combined'
    }
    df['dataset'] = df['dataset'].map(dataset_group_map)
    return df

def aggregate_by_new_groups(df):
    """Aggregate DataFrame by taking the mean over new dataset groups 'homo' and 'hetero'."""
    # Group by all columns except 'value', take mean, and reset index
    group_cols = [col for col in df.columns if col != metric and col != (metric + '_error')]
    group_cols = ['dataset', 'use_contrastive', 'algo', 'topology', 'edge_drop']
    print(group_cols)
    return df.groupby(group_cols).mean().reset_index()


In [127]:
def process_topology_df(df, error_type='std'):
    regex = r'^(?P<algo>modular|monolithic)_topology_(?P<topology>[a-zA-Z_]+)_edge_drop_(?P<edge_drop>\d+\.\d+)$'


    df = df[df['use_contrastive'] == False]

    df = cal_mean_and_error(df, metric=metric, error_type=error_type)
    df[['algo', 'topology', 'edge_drop']]= df['algo'].str.extract(regex)


    # concat topology and edge_drop to `algo`

    df['algo'] = df['algo'] + '_' + df['topology'] + '_' + df['edge_drop']
    return df

In [128]:
def process_vanilla_df(df, error_type='std'):
    df = df[df['use_contrastive'] == False]
    df = cal_mean_and_error(df, metric=metric, error_type=error_type)
    df['algo'] = df['algo'].apply(lambda x: x + ' (baseline)')

    df['edge_drop'] = 0.0
    df['topology'] = 'fully_connected'
    return df

In [129]:
def process_fc_fed_df(df, error_type='std'):
    df = df[df['use_contrastive'] == False]
    df = cal_mean_and_error(df, metric=metric, error_type=error_type)
    df['edge_drop'] = "0.0"
    df['topology'] = 'fully_connected'

    df['algo'] = df['algo'] + '_' + df['topology'] + '_' + df['edge_drop']
    return df

In [130]:
fc_fed_df = pd.read_csv(fc_fed_path + '.csv')
fc_fed_df = process_fc_fed_df(fc_fed_df, error_type=error_type)
fc_fed_df

,algo,use_contrastive,dataset,final_acc,final_acc_error,edge_drop,topology
0,modular_fully_connected_0.0,False,cifar100,0.604791,0.002477,0.0,fully_connected
1,modular_fully_connected_0.0,False,combined,0.811777,0.004411,0.0,fully_connected
2,modular_fully_connected_0.0,False,fashionmnist,0.924572,0.004518,0.0,fully_connected
3,modular_fully_connected_0.0,False,kmnist,0.800375,0.003481,0.0,fully_connected
4,modular_fully_connected_0.0,False,mnist,0.935689,0.001828,0.0,fully_connected
5,monolithic_fully_connected_0.0,False,cifar100,0.670453,0.001610,0.0,fully_connected
6,monolithic_fully_connected_0.0,False,combined,0.865661,0.003961,0.0,fully_connected
7,monolithic_fully_connected_0.0,False,fashionmnist,0.936761,0.003852,0.0,fully_connected
8,monolithic_fully_connected_0.0,False,kmnist,0.805530,0.003272,0.0,fully_connected
9,monolithic_fully_connected_0.0,False,mnist,0.938638,0.001979,0.0,fully_connected


In [131]:
fed_df = pd.read_csv(fedavg_path + '.csv')
fed_df = process_topology_df(fed_df, error_type=error_type)
fed_df = fed_df[fed_df['topology'] != 'random']

In [132]:
vanilla_df = pd.read_csv('experiment_results/vanilla_jorge_setting_basis_no_sparse.csv')
vanilla_df = process_vanilla_df(vanilla_df, error_type=error_type)
vanilla_df

,algo,use_contrastive,dataset,final_acc,final_acc_error,edge_drop,topology
0,modular (baseline),False,cifar100,0.700353,0.001357,0.0,fully_connected
1,modular (baseline),False,combined,0.885849,0.003560,0.0,fully_connected
2,modular (baseline),False,fashionmnist,0.928013,0.003087,0.0,fully_connected
3,modular (baseline),False,kmnist,0.806317,0.003094,0.0,fully_connected
4,modular (baseline),False,mnist,0.934668,0.002005,0.0,fully_connected
5,monolithic (baseline),False,cifar100,0.631084,0.003176,0.0,fully_connected
6,monolithic (baseline),False,combined,0.865895,0.003669,0.0,fully_connected
7,monolithic (baseline),False,fashionmnist,0.924673,0.003335,0.0,fully_connected
8,monolithic (baseline),False,kmnist,0.782273,0.003078,0.0,fully_connected
9,monolithic (baseline),False,mnist,0.919588,0.002693,0.0,fully_connected


In [133]:
df = pd.concat([fed_df, vanilla_df, fc_fed_df])

In [134]:
df_norm = normalize_by_baseline(df)

df_mapped = map_datasets_to_new_groups(df_norm.copy())



In [135]:
remap_name = {
    'modular_fully_connected_0.0': 'fully connected',
    'modular_random_disconnect_0.25': 'p=0.25',
    'modular_random_disconnect_0.5': 'p=0.5',
    'modular_random_disconnect_0.7': 'p=0.7',
    'modular_random_disconnect_0.9': 'p=0.9',
    'modular_ring_0.0': 'ring',
    'modular_server_0.0': 'server',
    'modular_tree_0.0': 'tree',
    'monolithic_fully_connected_0.0': 'fully connected',
    'monolithic_random_disconnect_0.25': 'p=0.25',
    'monolithic_random_disconnect_0.5': 'p=0.5',
    'monolithic_random_disconnect_0.7': 'p=0.7',
    'monolithic_random_disconnect_0.9': 'p=0.9',
    'monolithic_ring_0.0': 'ring',
    'monolithic_server_0.0': 'server',
    'monolithic_tree_0.0': 'tree'
}




In [136]:
color_map = {
    'modular_fully_connected_0.0': 'blue',
    'modular_random_disconnect_0.25': 'lightblue',
    'modular_random_disconnect_0.5': 'skyblue',
    'modular_random_disconnect_0.7': 'deepskyblue',
    'modular_random_disconnect_0.9': 'dodgerblue',
    'modular_ring_0.0': 'royalblue',
    'modular_server_0.0': 'mediumblue',
    'modular_tree_0.0': 'navy',
    'monolithic_fully_connected_0.0': 'red',
    'monolithic_random_disconnect_0.25': 'salmon',
    'monolithic_random_disconnect_0.5': 'lightcoral',
    'monolithic_random_disconnect_0.7': 'indianred',
    'monolithic_random_disconnect_0.9': 'crimson',
    'monolithic_ring_0.0': 'firebrick',
    'monolithic_server_0.0': 'darkred',
    'monolithic_tree_0.0': 'maroon'
}


In [137]:
edgecolor_map = {
    'modular_fully_connected_0.0': 'darkblue',
    'modular_random_disconnect_0.25': 'steelblue',
    'modular_random_disconnect_0.5': 'steelblue',
    'modular_random_disconnect_0.7': 'steelblue',
    'modular_random_disconnect_0.9': 'steelblue',
    'modular_ring_0.0': 'midnightblue',
    'modular_server_0.0': 'blue',
    'modular_tree_0.0': 'darkblue',
    'monolithic_fully_connected_0.0': 'darkred',
    'monolithic_random_disconnect_0.25': 'brown',
    'monolithic_random_disconnect_0.5': 'brown',
    'monolithic_random_disconnect_0.7': 'brown',
    'monolithic_random_disconnect_0.9': 'brown',
    'monolithic_ring_0.0': 'darkred',
    'monolithic_server_0.0': 'red',
    'monolithic_tree_0.0': 'darkred'
}


In [138]:
hatch_map = {
    'modular_fully_connected_0.0': '',    # Slashes
    'modular_random_disconnect_0.25': '//', # Vertical lines
    'modular_random_disconnect_0.5': '||',  # Vertical lines
    'modular_random_disconnect_0.7': '\\', # Vertical lines
    'modular_random_disconnect_0.9': '-', # Vertical lines
    'modular_ring_0.0': 'oo',               # Circles
    'modular_server_0.0': '++',             # Crosses
    'modular_tree_0.0': 'O', # Backslashes
    'monolithic_fully_connected_0.0': '', # Slashes
    'monolithic_random_disconnect_0.25': '//', # Vertical lines
    'monolithic_random_disconnect_0.5': '||',  # Vertical lines
    'monolithic_random_disconnect_0.7': '\\', # Vertical lines
    'monolithic_random_disconnect_0.9': '-', # Vertical lines
    'monolithic_ring_0.0': 'oo',               # Circles
    'monolithic_server_0.0': '++',             # Crosses
    'monolithic_tree_0.0': 'O', # Backslashes
}


In [139]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.collections import PatchCollection


plt.style.use('seaborn-whitegrid')


# define an object that will be used by the legend
class MulticolorPatch(object):
    def __init__(self, colors):
        self.colors = colors
        
# define a handler for the MulticolorPatch object
class MulticolorPatchHandler(object):
    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        width, height = handlebox.width, handlebox.height
        patches = []
        for i, c in enumerate(orig_handle.colors):
            patches.append(plt.Rectangle([width/len(orig_handle.colors) * i - handlebox.xdescent, 
                                          -handlebox.ydescent],
                           width / len(orig_handle.colors),
                           height, 
                           facecolor=c, 
                           edgecolor='none'))

        patch = PatchCollection(patches,match_original=True)

        handlebox.add_artist(patch)
        return patch



def plot(df, color_map, edgecolor_map, remap_name, name='', ax=None, include_legend=True):
    if ax is None:
        figsize = (15, 5)
        fig, ax = plt.subplots(figsize=figsize)

    # Plotting each bar with a different color
    legend_entries = {}
    for i, (algo, v, v_err) in enumerate(zip(df['algo'], df[metric], df[metric + '_error'])):
        if algo not in remap_name:
            continue
        bar = ax.bar(i, v, color=color_map[algo], label=remap_name[algo],
                    edgecolor=edgecolor_map[algo], yerr=v_err)
        label = remap_name[algo]
        if label not in legend_entries:
            legend_entries[label] = [None, None]  # [modular color, monolithic color]
        if 'modular' in algo:
            legend_entries[label][0] = color_map[algo]
        elif 'monolithic' in algo:
            legend_entries[label][1] = color_map[algo]

    ax.set_xticks([])
    ax.set_ylabel(f'{metric} improvement (%)', fontsize=14)
    ax.set_xlabel('Topology', fontsize=14)
    ax.set_title(name, fontsize=16)

    if include_legend:
        # Create custom legend entries
        print(legend_entries)
        ax.legend([MulticolorPatch(c) for c in legend_entries.values()], legend_entries.keys(), handler_map={MulticolorPatch: MulticolorPatchHandler()}, frameon=True, loc='lower right', bbox_to_anchor=(1.1, 0.0))

    ax.grid(True, which='major', linestyle='--', alpha=0.5)
    plt.tight_layout()

/tmp/ipykernel_144903/3974062340.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [140]:
df_heter2

,dataset,use_contrastive,algo,topology,edge_drop,auc,auc_error
0,cifar100,False,modular (baseline),fully_connected,0.0,0.000000,0.187350
1,cifar100,False,modular_fully_connected_0.0,fully_connected,0.0,-10.993766,0.269998
2,cifar100,False,modular_random_disconnect_0.25,random_disconnect,0.25,-10.954287,0.258476
3,cifar100,False,modular_random_disconnect_0.5,random_disconnect,0.5,-11.266626,0.306100
4,cifar100,False,modular_random_disconnect_0.7,random_disconnect,0.7,-11.340636,0.288424
5,cifar100,False,modular_random_disconnect_0.9,random_disconnect,0.9,-11.928305,0.314935
6,cifar100,False,modular_ring_0.0,ring,0.0,-11.224303,0.278630
7,cifar100,False,modular_server_0.0,server,0.0,-11.527160,0.310470
8,cifar100,False,modular_tree_0.0,tree,0.0,-11.381241,0.279664
9,cifar100,False,monolithic (baseline),fully_connected,0.0,0.000000,0.494235


['dataset', 'use_contrastive', 'algo', 'topology', 'edge_drop']
['dataset', 'use_contrastive', 'algo', 'topology', 'edge_drop']
['dataset', 'use_contrastive', 'algo', 'topology', 'edge_drop']
{'fully connected': ['blue', 'red'], 'p=0.25': ['lightblue', 'salmon'], 'p=0.5': ['skyblue', 'lightcoral'], 'p=0.7': ['deepskyblue', 'indianred'], 'p=0.9': ['dodgerblue', 'crimson'], 'ring': ['royalblue', 'firebrick'], 'server': ['mediumblue', 'darkred'], 'tree': ['navy', 'maroon']}


In [141]:
df_homo = df_mapped[df_mapped['dataset'] == 'homogenous']
df_homo = aggregate_by_new_groups(df_homo)
df_homo[metric] *= 100
df_homo[metric + '_error'] *= 100


# df_heter = df_mapped[df_mapped['dataset'] == 'heterogeneous']
df_heter = df_mapped[df_mapped['dataset'] == 'combined']
df_heter = aggregate_by_new_groups(df_heter)
df_heter[metric] *= 100
df_heter[metric + '_error'] *= 100


df_heter2 = df_mapped[df_mapped['dataset'] == 'cifar100']
df_heter2 = aggregate_by_new_groups(df_heter2)
df_heter2[metric] *= 100
df_heter2[metric + '_error'] *= 100


# fig, ax = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
fig, ax = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

plot(df_homo, color_map, edgecolor_map, remap_name, name='Homogeneous', ax=ax[0], include_legend=False)
# plot(df_heter, color_map, edgecolor_map, remap_name,  name='Heterogeneous', ax=ax[1])
plot(df_heter, color_map, edgecolor_map, remap_name,  name='Combined', ax=ax[1], include_legend=False)
plot(df_heter2, color_map, edgecolor_map, remap_name,  name='cifar100', ax=ax[2])

# fig.suptitle('Federated topology', fontsize=20, fontweight='bold');
fig.savefig('Figures/topology.pdf')
